In [1]:
import pybullet as p
import numpy as np
p.connect(p.GUI)
ur5 = p.loadURDF("ur5e.urdf",[0,0,0])

In [2]:
n_joints = p.getNumJoints(ur5)
n_joints

22

In [3]:

jointNames = [p.getJointInfo(ur5, i)[1] for i in range(0,n_joints)]
for i, n in enumerate(jointNames):
    print(i,n)

0 b'shoulder_pan_joint'
1 b'shoulder_lift_joint'
2 b'elbow_joint'
3 b'wrist_1_joint'
4 b'wrist_2_joint'
5 b'wrist_3_joint'
6 b'ee_fixed_joint'
7 b'wrist_3_link-tool0_fixed_joint'
8 b'tool_joint'
9 b'robotiq_2f_85_left_driver_mimic_joint'
10 b'robotiq_2f_85_left_coupler_joint'
11 b'robotiq_2f_85_left_spring_link_joint'
12 b'robotiq_2f_85_right_driver_mimic_joint'
13 b'robotiq_2f_85_right_coupler_joint'
14 b'robotiq_2f_85_right_spring_link_joint'
15 b'coupler_joint'
16 b'robotiq_ur_coupler_robotiq_2f_85_base_joint'
17 b'robotiq_2f_85_left_driver_joint'
18 b'robotiq_2f_85_left_pad_joint'
19 b'robotiq_2f_85_right_driver_joint'
20 b'robotiq_2f_85_right_pad_joint'
21 b'base_link-base_fixed_joint'


In [4]:
def close_gripper(percent):
    # left/ right driver appears to close at 0.03
    percent -= 0.2
    driver = percent*0.055
    p.resetJointState(ur5, 18, driver)
    p.resetJointState(ur5, 20, driver)
    spring_link = percent * 0.5
    p.resetJointState(ur5, 12, spring_link)
    p.resetJointState(ur5, 15, spring_link)
    driver_mimic = percent * 0.8
    p.resetJointState(ur5, 10, driver_mimic)
    p.resetJointState(ur5, 13, driver_mimic)
    
close_gripper(0)

In [5]:
joints_indices = [0,1,2,3,4,5]
controls= []
JOINTS = True

In [6]:
def add_xyz_rpy_controls():
    controls = []
    orn = [0,0,0]
    controls.append(p.addUserDebugParameter("X", -1, 1, 0))
    controls.append(p.addUserDebugParameter("Y", -1, 1, 0.00))
    controls.append(p.addUserDebugParameter("Z", -1, 1, 0.2))
    controls.append(p.addUserDebugParameter("R", -4, 4, orn[0]))
    controls.append(p.addUserDebugParameter("P", -4, 4, orn[1]))
    controls.append(p.addUserDebugParameter("Y", -4,4, orn[2]))
    controls.append(p.addUserDebugParameter("grip", 0, 1, 0))
    return controls

def add_joint_controls():
    for i in joints_indices:
        controls.append(p.addUserDebugParameter(str(i), -np.pi, np.pi, 0))
    
if JOINTS:
    add_joint_controls()
else:
    add_xyz_rpy_controls()

In [13]:
default_joints = [-0.9572998713387112,
                             -0.8639503472294617,
                             -1.9830303329043686,
                             -1.8654094162934607,
                             1.5707874376070445,
                             0.6135043943552552]

#default_joints  = [1.389, -1.058, 1.554, 1.124, 1.587, 0.562]
# sets the arm to the current joint positions so IK calcs are accurate
def set_states(states):
    for idx, i in enumerate(joints_indices):
        p.resetJointState(ur5, i, states[idx])
        
set_states(default_joints)


In [14]:
ee_index=6
while(1):
    poses = []
    for c in controls:
        poses.append(p.readUserDebugParameter(c))
    if JOINTS:
        for i in joints_indices:
            p.changeDynamics(ur5, i, linearDamping=0, angularDamping=0)
            p.resetJointState(ur5, i, poses[i])
        state = p.getLinkState(ur5, ee_index, computeLinkVelocity=1)
        orn = state[1]
        #print(p.getEulerFromQuaternion(orn))
    else:
        pass

KeyboardInterrupt: 

In [22]:
def calc_angles(pos,ori):
    for i in range(0,10): # converge on the solution
        angles = p.calculateInverseKinematics(ur5,ee_index, pos,ori)[0:6]
        set_states(angles)
        
    return p.calculateInverseKinematics(ur5,ee_index, pos,ori)[0:6]

def get_position():
    return p.getLinkState(ur5,ee_index)[0:2]



In [46]:
import time

In [25]:
set_states(default_joints)
pos, ori = get_position()
print(p.getEulerFromQuaternion(ori))
print(pos)

(0.0, 1.5707963267948966, 1.5707962985891812)
(-0.0050628002345312366, 0.23993969610998148, 0.5000003143099447)


In [52]:
default_ori = p.getQuaternionFromEuler([0,0,0])
default_pos = (0,0.4,0.3)

In [53]:
states = calc_angles(default_pos, default_ori)
set_states(states)

In [47]:
default_ori = p.getQuaternionFromEuler([0,np.pi/2,np.pi/2])
default_pos = (0,0.4,0.3)
states = calc_angles(default_pos, default_ori)
set_states(states)
time.sleep(1)
default_ori = p.getQuaternionFromEuler([0,np.pi,np.pi/2])
default_pos = (0,0.4,0.3)
states = calc_angles(default_pos, default_ori)
set_states(states)
time.sleep(1)
default_ori = p.getQuaternionFromEuler([0,np.pi,np.pi])
default_pos = (0,0.4,0.3)
states = calc_angles(default_pos, default_ori)
set_states(states)
time.sleep(1)
default_ori = p.getQuaternionFromEuler([0,np.pi/2,np.pi])
default_pos = (0,0.4,0.3)
states = calc_angles(default_pos, default_ori)
set_states(states)
time.sleep(1)
default_ori = p.getQuaternionFromEuler([0,0,np.pi/2])
default_pos = (0,0.4,0.3)
states = calc_angles(default_pos, default_ori)
set_states(states)
time.sleep(1)
default_ori = p.getQuaternionFromEuler([0,np.pi/2,0])
default_pos = (0,0.4,0.3)
states = calc_angles(default_pos, default_ori)
set_states(states)
time.sleep(1)

In [6]:
poses = []
for i in range(0, n_joints):
    poses.append(p.readUserDebugParameter(controls[i]))
for j in range(n_joints):
    info = p.getJointInfo(ur5, j)
        # print("info=",info)
    jointName = info[1]
    jointType = info[2]
    #print(jointName)
    p.changeDynamics(ur5, j, linearDamping=0, angularDamping=0)
    p.resetJointState(ur5, j, poses[j])

In [15]:
print(poses)

[0.37894725799560547, 0.6315789222717285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [4]:

joints_indices = [1,2,3,4,5,6]

In [6]:

jointNames = [p.getJointInfo(ur5, i)[1] for i in joints_indices]
jointNames

[b'shoulder_pan_joint',
 b'shoulder_lift_joint',
 b'elbow_joint',
 b'wrist_1_joint',
 b'wrist_2_joint',
 b'wrist_3_joint']

In [262]:

defaut_joints = [-0.9572998713387112,
                             -0.8639503472294617,
                             -1.9830303329043686,
                             -1.8654094162934607,
                             1.5707874376070445,
                             0.6135043943552552]
default_ori = p.getQuaternionFromEuler([np.pi,0,0])
default_pos = (0,0.0,0.5)
        
ee_index=9

In [19]:

# sets the arm to the current joint positions so IK calcs are accurate
def set_states(states):
    for idx, i in enumerate(joints_indices):
        p.resetJointState(ur5, i, states[idx])


def get_position():
    return p.getLinkState(ur5,ee_index)[0:2]
    
def calc_angles(pos,ori):
    for i in range(0,10): # converge on the solution
        angles = p.calculateInverseKinematics(ur5,ee_index, pos,ori)[0:6]
        set_states(angles)
        
    return p.calculateInverseKinematics(ur5,ee_index, pos,ori)[0:6]

In [46]:


calc_angles(pos, ori)